In [ ]:
"""
Go SHORT on MNQ (Sell 1 contract)
This will either: 
- Open a short position (if flat)
- Close a long position (if currently long 1)
"""
import os
import requests
from dotenv import load_dotenv

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"

print("=" * 60)
print("GO SHORT (SELL) - MNQ")
print("=" * 60)

# Authenticate
print("\n[1/3] Authenticating...")
auth_response = requests.post(
    f"{API_BASE}/Auth/loginKey",
    json={"userName": USERNAME, "apiKey": KEY},
    headers={"accept": "text/plain", "Content-Type": "application/json"},
    timeout=15
)

if auth_response.status_code != 200:
    print(f"❌ Auth failed")
    exit(1)

token = auth_response.json()["token"]
print(f"✅ Authenticated")

headers = {
    "Authorization": f"Bearer {token}",
    "accept": "text/plain",
    "Content-Type": "application/json"
}

# Get Account
print("\n[2/3] Getting account...")
search_response = requests.post(
    f"{API_BASE}/Account/search",
    json={"onlyActiveAccounts": True},
    headers=headers,
    timeout=15
)

account = search_response.json()["accounts"][0]
account_id = account["id"]
print(f"✅ Account: {account['name']}")
print(f"   Balance: ${account.get('balance', 0):,.2f}")

# Place SELL order
print("\n[3/3] Placing SELL order...")
print("Side: 1 (SELL)")

order_payload = {
    "accountId": account_id,
    "contractId": "CON.F.US.MNQ.H26",
    "type": 2,    # Market
    "side": 1,    # SELL
    "size": 1
}

print(f"Payload: {order_payload}")

response = requests.post(
    f"{API_BASE}/Order/place",
    json=order_payload,
    headers=headers,
    timeout=15
)

print(f"\nStatus Code: {response.status_code}")
result = response.json()
print(f"Response: {result}")

if result.get('success'):
    print(f"\n✅ SUCCESS! Order ID: {result.get('orderId')}")
    print("Position: Either SHORT 1 MNQ or FLAT (if you were long)")
else:
    print(f"\n❌ FAILED: {result.get('errorMessage')}")

print("\n" + "=" * 60)